In [ ]:
# this is to setup the path so we can import the mindpype library
import os; os.sys.path.append(os.path.dirname(os.path.abspath('.')))

In [ ]:
# import mindpype and numpy
import mindpype as mp
import numpy as np

In [ ]:
# create a MindPype session object
# a session is a container for all the mindpype data and graphs
sess = mp.Session.create()

In [ ]:
# create the graph object and define some parameters
online_graph = mp.Graph.create(sess)

Fs = 128
l_freq = 1
h_freq = 40
Nc = 14
Ns = int(Fs * 10)

In [ ]:
# create the filter object
f = mp.Filter.create_fir(sess, Fs, l_freq, h_freq, method='fir', phase='minimum')

In [ ]:
# create the source object
LSL_data_src = mp.source.InputLSLStream.create_marker_uncoupled_data_stream(sess, pred="type='EEG'", channels=np.arange(3,17), interval=2.0, Ns=Ns)
online_input_data = mp.Tensor.create_from_handle(sess, (Nc, Ns), LSL_data_src)

# create the virtual tensor edges
v_tensors = [
    mp.Tensor.create_virtual(sess),  # 0 - output of pad, input to filter
    mp.Tensor.create_virtual(sess),  # 1 - output of filter, input to extract
]

# define the output tensor
online_output_data = mp.Tensor.create(sess, (Nc, Ns))

In [ ]:
# add the nodes to the graph
# pad the data
mp.kernels.PadKernel.add_to_graph(
    online_graph,
    online_input_data,
    v_tensors[0],
    pad_width=((0, 0), (len(f.coeffs["fir"]), len(f.coeffs["fir"]))),
    mode="edge"
)

mp.kernels.FilterKernel.add_to_graph(
    online_graph, v_tensors[0], f, v_tensors[1], axis=1,
)

start_time = 0.0
end_time = 10.0
start_ix = int(start_time * Fs) + len(f.coeffs["fir"])
end_ix = int(np.ceil(end_time * Fs)) + len(f.coeffs["fir"])
extract_indices = [
    slice(None),
    slice(start_ix, end_ix),
]  # All epochs, all channels, start_time to end_time

mp.kernels.ExtractKernel.add_to_graph(
    online_graph, v_tensors[1], extract_indices, online_output_data
)

In [ ]:
# verify and initialize the graph
online_graph.verify()
online_graph.initialize() # there is nothing to initialize in this graph, so this will do nothing

In [ ]:
import time
import matplotlib.pyplot as plt
# change the plot size
plt.rcParams["figure.figsize"] = (12 ,8)

def plot_trial(X):
    fig, ax = plt.subplots()
    #ax.clear()
    t = np.arange(0, X.shape[1]/128, 1/128)
    for i_ch, ch in enumerate(range(X.shape[0])):
        ax.plot(t, X[i_ch,:]+i_ch*200, label=ch)
    ax.set_yticks(())
    ax.set_xlabel('Time (s)', fontsize=28)
    plt.show()

# execute the graph
while True:
    tic = time.time()
    online_graph.execute()
    toc = time.time()
    print(f"Execution time: {toc-tic}")
    plot_trial(online_output_data.data)

